# AWS Elastic Beanstalk Docker Deployment

Further documentation of the code can be found on the original post linked [here](https://medium.com/@lloyd.hamilton/deploying-docker-containerised-ml-models-on-aws-elastic-beanstalk-67cbfbb2def4).

This notebook contains code that:

1. Trains a random forest model
2. Builds a Flask API
3. Test a local Flask API
4. Test a deployed Flask API


#### Train a random forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.datasets import load_iris

def display_scores(scores):
    print("Scores:", scores)
    print("Means:", scores.mean())
    print("Standard Deviation:", scores.std())

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_scores = cross_val_score(rf_clf, X_train, y_train, scoring = 'accuracy', cv = 3)
display_scores(rf_scores)
joblib.dump(rf_clf, 'models/rf_clf.joblib')

#### Build a Flask API application

In [ ]:
%%writefile scripts/flask_api.py

import joblib
import numpy as np
from flask import Flask, request
from flask import Response
import json

app = Flask(__name__)

@app.route('/')
def home_endpoint():
    return 'Hello World!'

@app.route('/predict', methods=['POST'])
def predict_endpoint():
    test_data = request.get_json()
    test_data = np.array(test_data)
    predictions = model.predict(test_data)
    response = json.dumps(predictions.tolist())
    return Response(response, status=200, mimetype="application/json")

if __name__ == '__main__':
    model = joblib.load('rf_clf.joblib')
    app.run(host='0.0.0.0', port=8080)

#### Test a local Flask API

**Note:** Ensure the docker container is running before running the cell below.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import requests
import json

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

url = 'http://localhost:8080/predict'
headers = {
    'Content-type': "application/json"
}

data = json.dumps(X_test.tolist())
response = requests.post(url, headers=headers, data=data)
predictions = np.array(json.loads(response.text))
print(f'Predictions: {predictions}')
print(f'Accuracy Score: {accuracy_score(predictions, y_test)}')

#### Testing deployed Flask API

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import requests
import json

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

url = 'http://flask-api-dev.eu-west-2.elasticbeanstalk.com/predict'
headers = {
    'Content-type': "application/json"
}

data = json.dumps(X_test.tolist())
response = requests.post(url, headers=headers, data=data)
predictions = np.array(json.loads(response.text))
print(f'Predictions: {predictions}')
print(f'Accuracy Score: {accuracy_score(predictions, y_test)}')